In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import wave
import random

/Users/tacky0612/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def wave_plot(filename): #WAVの正規化とそれの吐き出し
#     open wave file
    wf = wave.open(filename,'r')
    channels = wf.getnchannels()

#     load wave data
    time = 1 #sec
    chunk_size =44100*time #1sec = 44100
    amp  = (2**8) ** wf.getsampwidth() / 2
    x = wf.readframes(chunk_size)   # バイナリ読み込み
    x = np.frombuffer(x,'int16') # intに変換
    x = x / amp                  # 振幅正規化(-1~1)
    x = x[::channels]　　　　　　　# stereo　→　monoral
    
    return x

In [20]:
def data_set(count,instrument_list,start_list,end_list): #データセットを作りたい（願望）
    # count = データセットとして取り出したい数 , instrument_list = "楽器"(str) , start_list =  始点指定, end_list = データセットの終点指定 
    K = len(instrument_list)
    waves = np.array([])
    labels = np.array([])
    
    for i in range(int(count)):
        
        N = random.randint(0,K-1)
        n = random.randint(start_list[N],end_list[N]) #ファイルの全範囲までの乱数生成
        wave = wave_plot("./" + instrument_list[N] + "/output/" + str(n) +".wav") # ランダムで指定楽器のWAVデータを配列に入れる
        waves = np.append(waves,wave,axis=0)
        label = np.zeros(K)                #1-of-Kのラベル生成
        label.put(N, 1)
        labels = np.append(labels,label,axis=0)
        num = len(wave)
        
    #np.ndarrayの整形
    waves = np.reshape(waves,(count,num))
    labels = np.reshape(labels,(count,K))
    
    return waves,labels

In [21]:
namelist = ["piano","bass"]
startlist_test = [100,100,100,100]
endlist_test = [1000,1000,1000,1000]
startlist_train = [0,0,0,0]
endlist_train = [99,99,99,99]

In [22]:
num_units = 1024

x = tf.placeholder(tf.float32, [None, 44100])

w1 = tf.Variable(tf.truncated_normal([44100, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
hidden1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units, 2]))
b0 = tf.Variable(tf.zeros([]))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

In [23]:
t = tf.placeholder(tf.float32, [None,2])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [25]:
i = 0
waves_test, labels_test = data_set(100,namelist ,startlist_test,endlist_test)
for _ in range(500):
    i += 1
    batch_xs, batch_ts = data_set(100,namelist ,startlist_train,endlist_train)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 10 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:waves_test, t: labels_test})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

Step: 10, Loss: 70.296204, Accuracy: 0.490000
Step: 20, Loss: 74.970901, Accuracy: 0.480000
Step: 30, Loss: 77.985947, Accuracy: 0.450000
Step: 40, Loss: 86.870018, Accuracy: 0.480000
Step: 50, Loss: 88.758514, Accuracy: 0.430000
Step: 60, Loss: 96.590836, Accuracy: 0.460000
Step: 70, Loss: 96.749481, Accuracy: 0.430000
Step: 80, Loss: 99.294373, Accuracy: 0.430000
Step: 90, Loss: 103.288147, Accuracy: 0.470000
Step: 100, Loss: 104.330170, Accuracy: 0.440000
Step: 110, Loss: 108.098923, Accuracy: 0.460000
Step: 120, Loss: 109.718399, Accuracy: 0.450000
Step: 130, Loss: 109.831062, Accuracy: 0.460000
Step: 140, Loss: 108.978470, Accuracy: 0.430000
Step: 150, Loss: 111.365860, Accuracy: 0.440000
Step: 160, Loss: 114.249664, Accuracy: 0.450000
Step: 170, Loss: 115.547798, Accuracy: 0.450000
Step: 180, Loss: 114.917755, Accuracy: 0.430000
Step: 190, Loss: 116.908936, Accuracy: 0.450000
Step: 200, Loss: 118.408356, Accuracy: 0.450000
Step: 210, Loss: 117.948204, Accuracy: 0.450000
Step: 220